In [ ]:
import sys

sys.path.append("../")

import tensorflow as tf
import os

from collections import defaultdict
from scipy.signal import triang

from tensorflow.keras.models import load_model
from lib_dolphin.audio import *
from lib_dolphin.sequential import *
from lib_dolphin.eval import *
from lib_dolphin.discrete import *

FFT_STEP     = 128
FFT_WIN      = 512
FFT_HI       = 230
FFT_LO       = 100

D            = FFT_WIN // 2 - FFT_LO - (FFT_WIN // 2 - FFT_HI)

MODEL_PATH = 'model_v2/'


decoder       = load_model(f'{MODEL_PATH}/decoder_nn_20221129.h5', custom_objects = {'Functional' : tf.keras.models.Model})
lab           = pkl.load(open(f"{MODEL_PATH}/labels.pkl", "rb"))
reverse       = {v:k for k, v in lab.items()}
label_mapping = pkl.load(open(f'{MODEL_PATH}/label_mapping.pkl', 'rb'))

In [ ]:
def spec(x):
    return spectrogram(x, FFT_LO, FFT_HI, FFT_WIN, FFT_STEP)            
        
def decode(x, decoder, label_mapping, reverse, smoothing=True, ignoire_echo=True):
    t, d = x.shape
    a = x.reshape((1,t,d,1))
    p = decoder.predict(a).reshape((a.shape[1], label_mapping.n + 1)) 

    if len(p) > NEURAL_SMOOTH_WIN and smoothing:
        for i in range(0, len(p[0])):
            window = triang(NEURAL_SMOOTH_WIN) / np.sum(triang(NEURAL_SMOOTH_WIN)) 
            p[:, i] = np.convolve(p[:, i], window, mode='same')
    p[:, 0] *= NEURAL_NOISE_DAMPENING
    for i in range(1, len(p[0])):
        dc = i2name(i, reverse, label_mapping)
        if dc in NEURAL_LABEL_DAMPENING:
            df = NEURAL_LABEL_DAMPENING[dc]
            print(f" ... dampen {dc} by {df}")
            p[:, i] *= df
    
    local_c = p.argmax(axis=1)
    local_p = p.max(axis=1)                    
    local_c = [reject(local_c[i], local_p[i], NEURAL_REJECT[i2name(local_c[i], reverse, label_mapping)])
               for i in range(len(local_c))]
    return local_c, p, local_p



NEURAL_NOISE_DAMPENING = 1.0
NEURAL_LABEL_DAMPENING = {
    'Uc' : 0.0,
    'Ea' : 0.0,
    'Eb' : 0.0,
    'Ec' : 0.0,
    'Ed' : 0.0,
    'Ee' : 0.0,
    'Ef' : 0.0,
    'Eg' : 0.0,
    'Eh' : 0.0,
}
NEURAL_REJECT= defaultdict(lambda: 0.03, {
    'Bc' : 0.01,
    'Be' : 0.01,
})
NEURAL_SMOOTH_WIN=64

PRINT_LAB = None
                 
p = np.zeros((1, 33))
path = '../test_encode/larger/'
for file in os.listdir(path):
    if file.endswith('.wav'):
        print(file)
        fp = f"{path}{file}"
        x = raw(fp)
        s = spec(x)
        plottable = spectrogram(x, 0, FFT_WIN // 2, FFT_WIN, FFT_STEP)
        dec, _p, local_p  = decode(s, decoder, label_mapping, reverse)
        
        if PRINT_LAB is not None:
            for a, b in zip(dec, local_p):
                if i2name(a, reverse, label_mapping).startswith(PRINT_LAB) or PRINT_LAB =='ALL':
                    print(f"\t{a} {b}")
        c    = compress_neural(dec, len(s), reverse, label_mapping)
        plot_neural(plottable, c, None)  
        p = np.vstack([p, _p])
        print(p.shape)

In [ ]:
bins = np.linspace(0, 0.2, 250)

for i in range(0, 32):
    dc = i2name(i, reverse, label_mapping)
    if dc.startswith('B'):
        x = p[i, :]
        x = x[x > 0]
        th = [1/33]
        plt.hist(x, bins, label=dc)
        plt.title(i2name(i, reverse,label_mapping))
plt.legend()
plt.show()



In [ ]:
for i in range(0, 33):
    print(i2name(i, reverse,label_mapping ))


In [ ]:
1/33